# **IMPORT LIBRARIES**

In [65]:
import random
import spacy
from spacy.training.example import Example
import pandas as pd

# **TRAINING OUR PARSER**

In [66]:
nlp = spacy.blank("en")

# Add the custom component to the pipeline
parser = nlp.add_pipe("parser")

TRAIN_DATA = [
    ("Find a cafe with few people", {"heads": [0, 2, 0, 5, 5, 2], "deps": ["ROOT", "-", "PLACE", "-", "QUANTITY", "ATTRIBUTE"]}),
    ("Find a hotel with many rooms near the beach", {"heads": [0, 2, 0, 5, 5, 2, 7, 7, 7], "deps": ["ROOT", "-", "PLACE","-","QUANTITY","ATTRIBUTE", "-", "-", "LOCATION"]}),
    ("I need a room with enough space", {"heads": [1, 1, 3, 1, 6, 6, 3], "deps": ["-", "ROOT", "-", "PLACE", "-", "QUANTITY", "ATTRIBUTE"]}),
    
    ("Show me the store that sells some unique products", {"heads": [0, 0, 3, 0, 3, 3, 8, 8, 3], "deps": ["ROOT", "-", "-", "PLACE", "-", "-", "QUANTITY", "-", "ATTRIBUTE"]}),
    ("Find a restaurant with all the menu items", {"heads": [0, 2, 0, 6, 7, 6, 7, 2], "deps": ["ROOT", "-", "PLACE", "-", "QUANTITY", "-", "PRODUCT", "ATTRIBUTE"]}),
    ("Show me the hostel with half the rooms decorated" , {"heads": [0, 0, 3, 0, 5, 7, 7, 3, 7], "deps": ["ROOT", "-", "-", "PLACE", "-", "QUANTITY", "-", "ATTRIBUTE", "-"]}),
    
    ("Find a restaurant with whole cakes", {"heads": [0, 2, 0, 5, 5, 2], "deps": ["ROOT", "-", "PLACE", "-", "QUANTITY", "ATTRIBUTE"]}),
    ("Show me the shop with enough supplies", {"heads": [0, 0, 3, 0, 6, 6, 3], "deps": ["ROOT", "-", "-", "PLACE", "-", "QUANTITY", "ATTRIBUTE"]}),
    ("Visit a library with numerous books", {"heads": [0, 2, 0, 5, 5, 2], "deps": ["ROOT", "-", "PLACE", "-", "QUANTITY", "ATTRIBUTE"]}),
    
    ("There are few clouds in the sky", {"heads":[1, 1, 3, 1, 4, 6, 4], "deps": ["-","ROOT","QUANTITY","ATTRIBUTE","-","-","PLACE"]}),
    ("Enough effort was put into achieving the goal", {"heads":[1,3,3,3,3,4,7,5],"deps":['QUANTITY','ATTRIBUTE','-','ROOT','-','-','-','PLACE']}),
    ("All the ingredients for the recipe are in the pantry", {"heads":[2,2,6,2,5,3,6,6,9,7],"deps":['QUANTITY','-','ATTRIBUTE','ROOT','-','-','-','-','-','PLACE']}),
    
    ("Many tourists visited the famous counteries", {"heads":[1,2,2,5,5,2],"deps":['QUANTITY','ATTRIBUTE','ROOT','-','-','PLACE']}),
    ("Show me the cafe with a few tables available", {"heads": [0, 0, 4, 0, 5, 5 , 5, 5 , 2 ], "deps": ["ROOT", "-", "-", "PLACE","-", "-", "QUANTITY", "ATTRIBUTE" , "ATTRIBUTE"]}),
    ("Find a restaurant with whole cakes and a dozen pastries",{"heads": [0, 2, 0, 0, 5, 5, 3, 6,6,4], "deps": ["ROOT", "-", "PLACE", "-", "QUANTITY", "ATTRIBUTE", "-", "-", "QUANTITY" , "ATTRIBUTE"]}),
]


for text, annotations in TRAIN_DATA:
    for dep in annotations.get('deps', []):
        parser.add_label(dep)  

n_iter = 30	

# Training loop with multiple iterations
optimizer = nlp.begin_training()
for iteration in range(n_iter):
    # Shuffle the training data
    random.shuffle(TRAIN_DATA)
    losses = {}

    for text, annotations in TRAIN_DATA:
        # Create an Example object
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        # Update the model
        nlp.update([example],losses=losses, drop=0.2)     

# **TESTING THE MODEL**

In [68]:
# After training, you can test the model
tests = [
    "find a hotel with many rooms ",
    "show me the closest gym that's open late with enough space",
    "show me the store that sells some unique products",
    "find a restaurant with all the menu items",
    "show me the hostel with half the rooms decorated",
    "find a restaurant with whole cakes",
    "show me the shop with enough supplies near work",
    "visit a library with numerous books",
]

docs = nlp.pipe(tests)
for doc in docs:
    df = pd.DataFrame([(t.text, t.dep_, t.head.text) for t in doc if t.dep_ != "-"], columns=['text', 'dep', 'head'])
    display(df)

,text,dep,head
0,find,ROOT,find
1,hotel,PLACE,find
2,many,QUANTITY,rooms
3,rooms,ATTRIBUTE,hotel


,text,dep,head
0,show,ROOT,show
1,closest,PLACE,show
2,that,ATTRIBUTE,closest
3,late,PLACE,show
4,enough,QUANTITY,space
5,space,ATTRIBUTE,late


,text,dep,head
0,show,ROOT,show
1,store,PLACE,show
2,some,QUANTITY,products
3,products,ATTRIBUTE,store


,text,dep,head
0,find,ROOT,find
1,restaurant,PLACE,find
2,with,dep,items
3,all,QUANTITY,items
4,menu,PRODUCT,items
5,items,ATTRIBUTE,restaurant


,text,dep,head
0,show,ROOT,show
1,hostel,PLACE,show
2,half,QUANTITY,rooms
3,rooms,ATTRIBUTE,hostel


,text,dep,head
0,find,ROOT,find
1,restaurant,PLACE,find
2,whole,QUANTITY,cakes
3,cakes,ATTRIBUTE,restaurant


,text,dep,head
0,show,ROOT,show
1,shop,PLACE,show
2,enough,QUANTITY,supplies
3,supplies,ATTRIBUTE,shop
4,work,ATTRIBUTE,supplies


,text,dep,head
0,visit,ROOT,visit
1,library,PLACE,visit
2,numerous,QUANTITY,books
3,books,ATTRIBUTE,library
